#### Aprendizado por reforço ou Reinforcement Learning(RL) utilizando o método Q-Learning com o ambiente simulado Gym

Q-Learning é um algoritmo de aprendizado por reforço que visa aprender a função de valor Q(s,a) onde "s" é um estado e "a" é uma ação. O objetivo do Q-Learning é encontrar a política ótima que maximiza a recompensa acumulada esperada.

Gym, desenvolvido pela OpenAI, é uma biblioteca que fornece ambientes de simulação para experimentação com algoritmos de aprendizado por reforço. Ele fornece uma interface padronizada para uma variedade de ambientes, facilitando o desenvolvimento e teste de algoritmos de RL.

#### Será realizado o problema do Táxi, onde o táxi precisa aprender a levar passageiros de um local (R, G, B, Y) para o outro

* As letras em azul é o local de origem e as letras rosa é o local de destino
* O táxi fica amarelo quando está vazio e verde quando está com um passageiro

In [12]:
import gym
import random
import numpy as np

In [13]:
random.seed(1234)

streets = gym.make("Taxi-v3", render_mode='ansi').env 
streets.reset()
print("\n" + streets.render())


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




* O táxi pode andar nas 4 direções cardeais onde estiver ":" e não pode andar onde estiver "|"
* O algoritimo Q-Learning dará ao táxi 20 pontos quando ele alcançar sucesso, descontará 10 pontos caso pegue ou deixe o passageiro em uma possição ilegal e descontará 1 ponto por campo andado com o passgeiro

In [14]:
initial_state = streets.encode(2,3,2,0)
streets.s = initial_state
print("\n" + streets.render())


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [15]:
streets.P[initial_state]

{0: [(1.0, 368, -1, False)],
 1: [(1.0, 168, -1, False)],
 2: [(1.0, 288, -1, False)],
 3: [(1.0, 248, -1, False)],
 4: [(1.0, 268, -10, False)],
 5: [(1.0, 268, -10, False)]}

Cada linha corresponde a uma possivel ação:
* Mover em uma das 4 direções;
* Pegar ou deixar o passageiro.

Os 4 valores nas linhas indicam:
* A probabilidade de acatar a ação;
* O próximo estado da ação;
* A recompensa da ação;
* Se a ação é um local bem sucedido para deixar o passageiro

#### Hora de treinar nosso Táxi!

Inicialmente, a tabela q_table é preenchida com zeros, indicando que o valor esperado de qualquer par estado-ação é desconhecido, conforme o agente interage com o ambiente, esses valores são atualizados com base nas recompensas recebidas e nas estimativas das recompensas futuras.
Durante o treinamento, o agente usa a tabela q_table para escolher ações baseadas na política derivada dos valores Q(s,a).

In [52]:
q_table = np.zeros([streets.observation_space.n, streets.action_space.n]) 

print("número de diferentes situações ou posições que o agente pode encontrar no ambiente: ", streets.observation_space.n)
print("número de escolhas que o agente pode fazer em qualquer estado: ", streets.action_space.n)
print()

learning_rate = 0.1 # velocidade de treinamento
discount_factor = 0.6 # determina o quanto o algoritmo valoriza as recompensas a longo prazo em comparação com as recompensas imediatas
exploration = 0.1 # 10% de chance de fazer uma escolha aleatoria ao inves de sequir o valor Q
epochs = 10000 # irá acontecer 10.000 simulações

for run in range(epochs):
    state = streets.reset()[0] # para pegar apenas o estado atual
    # print(state)
    done = False
    
    while not done:
        
        # print(q_table)
        random_value = random.uniform(0,1)
        if(random_value < exploration):
            action = streets.action_space.sample() # explora qualquer ação aleatória
        else:
            action = np.argmax(q_table[state]) # escolhe a ação com o maior valor Q
        
        # print(streets.step(action))    
        next_state, reward, done, parameter, info = streets.step(action)
        
        prev_q = q_table[state, action]
        next_max_q = np.max(q_table[next_state])
        new_q = (1 - learning_rate) * prev_q + learning_rate * (reward + discount_factor * next_max_q) # formula do valor Q
        q_table[state, action] = new_q

        state = next_state

número de diferentes situações ou posições que o agente pode encontrar no ambiente:  500
número de escolhas que o agente pode fazer em qualquer estado:  6



In [53]:
q_table[initial_state] 

array([-2.42292084, -2.40486172, -2.40083173, -2.3639511 , -7.30794855,
       -6.36084921])

In [58]:
from IPython.display import clear_output
from time import sleep

for tripnum in range(1,5):
    state = streets.reset()[0]
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        next_state, reward, done, parameter, info = streets.step(action)
        clear_output(wait=True)
        print("Trip number: " + str(tripnum))
        print(streets.render())
        sleep(.5)
        state = next_state
    sleep(2)

Trip number: 4
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

